In [ ]:
# 获取股票行情数据
"""
"time"                #时间戳
"open"                #开盘价
"high"                #最高价
"low"                 #最低价
"close"               #收盘价
"volume"              #成交量
"amount"              #成交额
"settle"              #今结算
"openInterest"        #持仓量
"""

In [ ]:
from typing import List
import pandas as pd

# 日期工具
from datetime import datetime, timedelta
from xtquant import xtdata

In [ ]:
"""
get_market_data函数返回数据格式为：
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""


# 公共解析数据的方法，解析后还是一个字典类型dict，key为 stock_code，value为一个pandas DataFrame,行索引为时间戳，列索引为字段名
def parse_stock_data(stock_data: dict) -> dict:
    parsed_data = {}
    for stock_code, data in stock_data.items():
        if isinstance(data, pd.DataFrame):
            # 确保数据是DataFrame类型
            parsed_data[stock_code] = data
        else:
            print(f"警告：股票 {stock_code} 的数据不是DataFrame类型，实际类型为 {type(data)}")
    return parsed_data


In [ ]:
# 获取沪深A股中所有成分股的列表
sector_name: str = '沪深A股'
stock_list_in_sector: List[str] = xtdata.get_stock_list_in_sector(sector_name)
print(f'获取板块成分股列表{len(stock_list_in_sector)}成功：{stock_list_in_sector}')

In [ ]:
# 测试下载000001.SZ的日 k 行情数据并获取打印
stock_code: str = '000001.SZ'
print(f'开始下载股票{stock_code}的行情数据')
# 获取一年前的日期,yyyyMMdd格式
one_years_ago = (datetime.now() - timedelta(days=1 * 365)).strftime('%Y%m%d')
print(f'一年前的日期为：{one_years_ago}')
# 获取今天日期
today = datetime.now().strftime('%Y%m%d')
print(f'今天的日期为：{today}')
# 下载股票的日K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的日 K 数据成功")
# 下载股票的周K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1w',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的周 K 数据成功")
# 下载股票的月K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1M',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的月 K 数据成功")

In [ ]:
# 测试获取下载的行情数据--获取最近10条数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')
print(f'获取股票 {stock_code} 的日K行情测试数据成功：{stock_data}')
# 解析返回数据
"""
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""
if isinstance(stock_data, dict):
    for field, value in stock_data.items():
        print(f'字段 {field} 的数据类型为：{type(value)}')
        print(f'字段 {field} 的数据内容为：{value}')
else:
    print(f'获取的股票行情数据格式不正确，应该是字典类型，但实际为：{type(stock_data)}')

In [ ]:
# 获取下载的行情数据--获取所有数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=-1,
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')
print(f'获取股票 {stock_code} 的日K行情数据成功：{stock_data}')


In [ ]:
# 测试未下载的股票行情数据能否获取
stock_code = '000002.SZ'  # 测试未下载的股票代码
print(f'开始获取股票 {stock_code} 的行情数据')
xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)

In [ ]:
# 测试能否够获取前复权的股票行情数据
print(f"开始获取股票 {stock_code} 的前复权行情数据")
xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    dividend_type='front',  # 获取前复权数据
    fill_data=False
)

In [ ]:
# 验证能否获取到未执行下载的日期范围的行情数据，例如获取 两年前，两个月前，两天前，两年前到两个月前的行情数据
# 获取两年前的日期,yyyyMMdd格式
two_years_ago = (datetime.now() - timedelta(days=2 * 365)).strftime('%Y%m%d')
print(f'两年前的日期为：{two_years_ago}')
# 获取两个月前的日期,yyyyMMdd格式
two_months_ago = (datetime.now() - timedelta(days=2 * 30)).strftime('%Y%m%d')
print(f'两个月前的日期为：{two_months_ago}')
# 获取两天前的日期,yyyyMMdd格式
two_days_ago = (datetime.now() - timedelta(days=2)).strftime('%Y%m%d')
print(f'两天前的日期为：{two_days_ago}')
# 获取两年前到两个月前的所有日K 行情数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=two_years_ago,
    end_time=two_months_ago,
    count=-1,  # 获取所有数据
    dividend_type='none',  # 不获取分红派息数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的两年前到两个月前的行情数据格式：{type(stock_data)}')
# 打印获取到的两年前到两个月前的日K 行情数据
print(f'获取股票 {stock_code} 的两年前到两个月前的日K行情数据{len(stock_data)}成功：{stock_data}')